# Preparación de datos sobre consumo de agua

Secretaría de Cultura, Recreación y Deporte | Dirección Observatorio y Gestión del Conocimiento Cultural |
Sistemas de Información y Narrativas

7 de marzo de 2025

- Este notebook muestra el proceso de lectura, procesamiento y preparación de datos de consumo de agua residencial facturado en Bogotá, que fueron entregados por la EAAB en febrero de 2025 con el propósito de ser utilizados como insumo y referencia para el diseño de la estrategia de cambio comportamental de la ciudadanía asociado a la duración del tiempo de ducha.
- Con estos datos se construyó tablero de visualización de datos en looker studio, disponible en el siguiente enlace: [2025 Resumen consumo agua](https://lookerstudio.google.com/reporting/bc22d13c-e22b-45ab-9d4e-80e4b1d7979e/page/p_hzmzghpmld)

In [2]:
import pandas as pd

In [3]:
folder = 'datos'
file_usuarios = f'{folder}/CuentasInformacionTerritorio.parquet'
file_consumo = f'{folder}/CuentasConsumoProrrateado.parquet'

In [4]:
# Diccionario para mapear los nombres de los meses en español a números
meses_dict = {
    "Enero": "01", "Febrero": "02", "Marzo": "03", "Abril": "04",
    "Mayo": "05", "Junio": "06", "Julio": "07", "Agosto": "08",
    "Septiembre": "09", "Octubre": "10", "Noviembre": "11", "Diciembre": "12"
}

In [5]:
df_usuarios = pd.read_parquet(file_usuarios)
df_consumo = pd.read_parquet(file_consumo)

In [6]:
# Extraer el nombre del mes y el año
df_consumo["Mes"] = df_consumo["MesConsumo"].str.extract(r'([a-zA-Z]+)')  # Extrae el mes (parte textual)
df_consumo["Anio"] = df_consumo["MesConsumo"].str.extract(r'(\d{4})')  # Extrae el año (parte numérica)

# Mapear el nombre del mes a su número
df_consumo["Mes_Numero"] = df_consumo["Mes"].map(meses_dict)

# Crear la nueva columna con formato YYYY-MM
df_consumo["MesConsumo_YYYYMM"] = df_consumo["Anio"] + "-" + df_consumo["Mes_Numero"]

# Eliminar columnas auxiliares si ya no son necesarias
df_consumo.drop(columns=["Mes", "Mes_Numero"], inplace=True)

In [7]:
# Unir los dataframes usando la columna en común
df_consumo_total = pd.merge(df_usuarios, df_consumo, left_on='cc_cta_contrato', right_on='CuentaContrato', how='inner')
df_consumo_total.head()

,cc_cta_contrato,cc_clase_uso_desc,cc_estrato,ps_uhb,ps_unh,sg_latitud,sg_longitud,cod_distrito_hidraulico,nom_localidad,cod_localidad,cod_upl,nom_upl,cod_upz,nom_upz,CuentaContrato,MesConsumo,ConsumoM3,Anio,MesConsumo_YYYYMM
0,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,12615666,Diciembre2023,9.110169,2023,2023-12
1,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,12615666,Junio2024,1.393443,2024,2024-06
2,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,12615666,Marzo2024,6.606557,2024,2024-03
3,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,12615666,Septiembre2024,3.559322,2024,2024-09
4,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,12615666,Mayo2023,2.354430,2023,2023-05


In [8]:
df_consumo_total.drop(columns=["CuentaContrato","MesConsumo"], inplace=True)
df_consumo_total.head()

,cc_cta_contrato,cc_clase_uso_desc,cc_estrato,ps_uhb,ps_unh,sg_latitud,sg_longitud,cod_distrito_hidraulico,nom_localidad,cod_localidad,cod_upl,nom_upl,cod_upz,nom_upz,ConsumoM3,Anio,MesConsumo_YYYYMM
0,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,9.110169,2023,2023-12
1,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,1.393443,2024,2024-06
2,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,6.606557,2024,2024-03
3,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,3.559322,2024,2024-09
4,12615666,Residencial,3.0,1,0.0,4.59711,-74.16093,5010600,BOSA,07,UPL18,Kennedy,UPZ049,APOGEO,2.354430,2023,2023-05


In [9]:
# Calcular el consumo residencial de agua en m3 para cada cuenta en proporción al número de unidades habitacionales

# Evitar divisiones por cero reemplazando valores nulos con 0
df_consumo_total['ps_uhb'] = df_consumo_total['ps_uhb'].fillna(0)
df_consumo_total['ps_unh'] = df_consumo_total['ps_unh'].fillna(0)

# Calcular el total de unidades
df_consumo_total['total_unidades'] = df_consumo_total['ps_uhb'] + df_consumo_total['ps_unh']

# Evitar divisiones por cero asignando 0 cuando el total de unidades es 0
df_consumo_total['ConsumoM3Residencial'] = df_consumo_total.apply(
    lambda row: (row['ConsumoM3'] * row['ps_uhb'] / row['total_unidades']) if row['total_unidades'] > 0 else 0,
    axis=1
)

In [10]:
df_consumo_total.shape[0]

49545138

In [11]:
# Agrupar y aplicar las funciones de agregación
df_consumo_resumen = df_consumo_total.groupby(
    ['cc_clase_uso_desc', 'MesConsumo_YYYYMM', 'cc_estrato','cod_localidad','nom_upl','nom_upz'], as_index=False
).agg({
    'ps_uhb': 'sum',
    'ps_unh': 'sum',
    'ConsumoM3': 'sum',
    'ConsumoM3Residencial': 'sum',
    'sg_latitud': 'mean',
    'sg_longitud': 'mean'
})

df_consumo_resumen.head()

,cc_clase_uso_desc,MesConsumo_YYYYMM,cc_estrato,cod_localidad,nom_upl,nom_upz,ps_uhb,ps_unh,ConsumoM3,ConsumoM3Residencial,sg_latitud,sg_longitud
0,Especial,2023-01,1.0,01,Toberín,SAN CRISTOBAL NORTE,0,1.0,4.133333,0.0,4.742841,-74.016660
1,Especial,2023-01,1.0,02,Cerros Orientales,PARDO RUBIO,0,1.0,590.884316,0.0,4.636914,-74.055084
2,Especial,2023-01,1.0,02,Chapinero,PARDO RUBIO,0,1.0,13.433333,0.0,4.635862,-74.059800
3,Especial,2023-01,1.0,05,Cerros Orientales,LA FLORA,0,1.0,1.540150,0.0,4.498461,-74.082450
4,Especial,2023-01,1.0,05,Rafael Uribe,DANUBIO,0,6.0,573.762712,0.0,4.536705,-74.113508


In [12]:
# Agrupar y aplicar las funciones de agregación
df_consumo_localidad = df_consumo_total.groupby(
    ['Anio','MesConsumo_YYYYMM', 'cod_localidad'], as_index=False
).agg({
    'ps_uhb': 'sum',
    'ps_unh': 'sum',
    'ConsumoM3': 'sum',
    'ConsumoM3Residencial': 'sum'
})

df_consumo_localidad.head()

,Anio,MesConsumo_YYYYMM,cod_localidad,ps_uhb,ps_unh,ConsumoM3,ConsumoM3Residencial
0,2023,2023-01,01,217642,20354.0,2.373822e+06,1.930119e+06
1,2023,2023-01,02,80932,24852.0,1.088594e+06,6.430154e+05
2,2023,2023-01,03,34089,9704.0,4.063614e+05,2.279913e+05
3,2023,2023-01,04,107690,6257.0,9.461677e+05,8.259705e+05
4,2023,2023-01,05,103135,4757.0,8.635760e+05,8.032048e+05


In [13]:
import pandas as pd

# Guardar en un archivo Excel con dos hojas
with pd.ExcelWriter(f'{folder}/consumo_resumen_total.xlsx', engine="openpyxl", mode="w") as writer:
    df_consumo_resumen.to_excel(writer, sheet_name="consumo", index=False)
    df_consumo_localidad.to_excel(writer, sheet_name="consumo_localidad", index=False)

print("Archivo 'consumo_resumen_total.xlsx' guardado con éxito.")

Archivo 'consumo_resumen_total.xlsx' guardado con éxito.
